# Factorio配方配平建模

In [1]:
import numpy as np
from scipy import optimize as op
import sympy as sp
import pandas as pd
from IPython.display import display, Image, Math, Markdown

np.set_printoptions(precision=6, suppress=True, floatmode='maxprec')
pd.set_option('display.precision', 2)

# 数学模型
- 每个配方均可写成向量 $V_i = -a_i \cdot \langle x_i \rangle + b_i \cdot \langle y_i \rangle$ 的形式，其中基向量$\langle x_i \rangle$为原料，$\langle y_i \rangle$为产物，系数向量$a_i,b_i$每项均大于等于0
- 则系统总输入输出可写成各个配方的线性组合 $S = \sum k_i V_i, k_i \ge 0$ 
- 重新定义基向量$\langle y \rangle = \cup \langle y_i \rangle, \langle x \rangle = \cup \langle x_i \rangle - \langle y \rangle$，并以此重写每个配方向量$V_i = -a_i \cdot \langle x \rangle + b_i \cdot \langle y \rangle$ 和总输入输出 $S = \sum k_i V_i = -\sum k_i a_i \cdot \langle x \rangle + \sum k_i b_i \cdot \langle y \rangle = -A_K \cdot \langle x \rangle + B_K \cdot \langle y \rangle$
- 则配平目标可写为最优化系数向量$\mathop{\arg\min}_{\{k_i\}}(K+\lambda_1 A_K + \lambda_2 B_K)$，约束条件$B_K \ge B_0, A_K \ge 0, K \ge 0$，其中$B_0$为系统所需的最小产出
- 写为$\langle x, y \rangle$上的矩阵形式：
$$ 
\begin{align}
V & = \begin{bmatrix} -A | B \end{bmatrix} \\
S = K^T \cdot V & = \begin{bmatrix} -K^T \cdot A | K^T \cdot B \end{bmatrix}
\end{align}
$$
则
$$
\begin{align}
K_{target} & = \mathop{\arg\min}_{K}(\lambda_0 K + \lambda_1 A^T K + \lambda_2 B^T K), given \\
-A^T K & \le 0 \\
-B^T K & \le B_0 \\
0 & \le K \\
\end{align}
$$

# 实例

## 1. 石油工业平衡问题
设
$$\langle x \rangle = \langle 原油,水 \rangle, \langle y \rangle = \langle 重油,轻油,天然气 \rangle $$
已知各配方（工厂）每秒基础产量
$$
V = \begin{bmatrix} 高等原油处理 \\ 重油分解 \\ 轻油分解 \end{bmatrix} =
\begin{bmatrix}
 -20 & -10 & 5 & 9 & 11 \\
 0 & -15 & -20 & 15 & 0 \\
 0 & -15 & 0 & -15 & 10
\end{bmatrix} \cdot \langle x, y \rangle
$$
设每秒至少需输出 $B_0 = \begin{bmatrix} 10 & 10 & 40 \end{bmatrix} \cdot \langle y \rangle$，求各配方的倍率（工厂数量）$K = \begin{bmatrix} k_1, k_2, k_3 \end{bmatrix}$:

In [76]:
A = np.array([[20,10],[0,15],[0,15]])
B = np.array([[5, 9, 11],[-20,15,0],[0,-15,10]])
A_ub = -np.vstack([A.T, B.T])
b_ub = -np.array([0, 0, 10, 10, 40])

In [77]:
c = np.array([1,1,1]) # c = \lambda_0，工厂数最小
K = op.linprog(c, A_ub, b_ub)
print(f'倍率：{K.x}\n原料：{A.T.dot(K.x)}\n产物：{B.T.dot(K.x)}')

倍率：[3.636364 0.       0.      ]
原料：[72.727273 36.363636]
产物：[18.181818 32.727273 40.      ]


In [78]:
c = np.array([1,0]).dot(A.T) # c = \lambda_1，原料最少
K = op.linprog(c, A_ub, b_ub)
print(f'倍率：{K.x}\n原料：{A.T.dot(K.x)}\n产物：{B.T.dot(K.x)}')

倍率：[2.649573 0.162393 1.08547 ]
原料：[52.991453 45.213675]
产物：[10. 10. 40.]


In [79]:
c = np.array([1,1,1]).dot(B.T) # c = \lambda_2, 产物最接近目标
K = op.linprog(c, A_ub, b_ub)
print(f'倍率：{K.x}\n原料：{A.T.dot(K.x)}\n产物：{B.T.dot(K.x)}')

倍率：[2.649573 0.162393 1.08547 ]
原料：[52.991453 45.213675]
产物：[10. 10. 40.]


## 2. 岩浆副产品问题
设
$$\langle x \rangle = \langle 岩浆, 方解石 \rangle, \langle y \rangle = \langle 熔融铁, 熔融铜, 石矿 \rangle $$
已知各配方（工厂）单位时间基础产量
$$
V_i = \begin{bmatrix} 岩浆制熔融铁 \\ 岩浆制熔融铜 \\ 销毁熔融铜 \\ 销毁石矿 \end{bmatrix} = 
\begin{bmatrix}
 -500 & -1 & 250 & 0 & 10 \\
 -500 & -1 & 0 & 250 & 15 \\
 0 & 0 & 0 & -1000 & 0 \\
 0 & 0 & 0 & 0 & -1000 \\
\end{bmatrix} \cdot \langle x, y \rangle
$$
设单位时间至少需输出 $\begin{bmatrix} B_{0,1} & B_{0,2} & B_{0,3} \end{bmatrix} \cdot \langle y \rangle$，求各配方的倍率（工厂数量）$K = \begin{bmatrix} k_1, k_2, k_3 \end{bmatrix}$:

In [80]:
A = np.array([[500,1],[500,1],[0,0],[0,0]])
B = np.array([[250, 0, 10],[-0,250,15],[0,-1000,0],[0,0,-1000]])
A_ub = -np.vstack([A.T, B.T])
# b0 = np.array([b01,b02,b03])
# b_ub = -np.hstack([[0, 0], b0])

c = np.array([1,1,1]).dot(B.T) # c = \lambda_2，产物最接近目标

In [98]:
# 石矿不足
b0 = np.array([20000, 5000, 1200]) # 目标
b_ub = -np.hstack([[0, 0], b0])
K = op.linprog(c, A_ub, b_ub)
print(f'倍率：{K.x}\n原料：{A.T.dot(K.x)}\n产物：{B.T.dot(K.x)}')

倍率：[80.       26.666667  1.666667  0.      ]
原料：[53333.333333   106.666667]
产物：[20000.  5000.  1200.]


In [99]:
# 石矿溢出
b0 = np.array([5000, 20000, 1200]) # 目标
b_ub = -np.hstack([[0, 0], b0])
K = op.linprog(c, A_ub, b_ub)
print(f'倍率：{K.x}\n原料：{A.T.dot(K.x)}\n产物：{B.T.dot(K.x)}')

倍率：[20.  80.   0.   0.2]
原料：[50000.   100.]
产物：[ 5000. 20000.  1200.]
